<a href="https://colab.research.google.com/github/BirukovAlex/neto_Python/blob/main/%D0%94%D0%97_%D1%81%D1%82%D1%80%D0%B0%D1%85%D0%BE%D0%B2%D0%BA%D0%B0_ML_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предсказываем стоимость мед страховки



## Загужаем необходимые библиотеки

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Загружаем данные

Собраны данные:
- age: возраст
- sex: пол
- bmi: индекс массы тела
- children: количество детей, охваченных медицинским страхованием / количество иждивенцев
- smoker: курение
- region: регион (northeast, southeast, southwest, northwest).
- charges: индивидуальные медицинские расходы (его и хотим предсказать)

In [ ]:
# Загрузите данные из файла insurance.csv в переменную df
df=pd.read_csv('insurance.csv')


In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Смотрим статистику, что нет пропусков и отсуствующих значений

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [ ]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Преобразуем строковые данные

In [ ]:
# Заменяем пол и курение на числа
df['sex']=df['sex'].map({'male':1, 'female':0})
df['smoker']=df['smoker'].map({'yes':1,'no':0})

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [ ]:
# Заменяем регион на набор отдельных колонок (is_southwest, is_southeast и тп)
df = pd.get_dummies(df, columns=['region'])
df.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0


## Формируем признаки и целевую переменную

In [ ]:
## Сформируйте признаки и целевую переменную
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'charges',
       'region_northeast', 'region_northwest', 'region_southeast',
       'region_southwest'],
      dtype='object')

In [ ]:
X=df[['age', 'sex', 'bmi', 'children', 'smoker', 'region_northeast', 'region_northwest', 'region_southeast',
       'region_southwest']]
y = df['charges']

In [ ]:
X

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,0,0,0,1
1,18,1,33.770,1,0,0,0,1,0
2,28,1,33.000,3,0,0,0,1,0
3,33,1,22.705,0,0,0,1,0,0
4,32,1,28.880,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,0,1,0,0
1334,18,0,31.920,0,0,1,0,0,0
1335,18,0,36.850,0,0,0,0,1,0
1336,21,0,25.800,0,0,0,0,0,1


In [ ]:
y

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1338, dtype: float64

## Разделяем данные на выборку для обучения/проверки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_test

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
430,19,1,33.100,0,0,0,0,0,1
224,42,1,24.640,0,1,0,0,1,0
1103,58,1,36.080,0,0,0,0,1,0
1100,33,0,19.095,2,1,1,0,0,0
762,33,1,27.100,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...
1021,22,0,31.020,3,1,0,0,1,0
583,32,0,23.650,1,0,0,0,1,0
1206,59,0,34.800,2,0,0,0,0,1
618,19,0,33.110,0,1,0,0,1,0


In [ ]:
X_train

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
335,64,1,34.500,0,0,0,0,0,1
662,32,0,31.540,1,0,1,0,0,0
3,33,1,22.705,0,0,0,1,0,0
530,57,1,42.130,1,1,0,0,1,0
712,43,0,30.685,2,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
610,47,0,29.370,1,0,0,0,1,0
1106,49,0,29.925,0,0,0,1,0,0
1060,24,1,32.010,0,0,0,0,1,0
733,48,0,27.265,1,0,1,0,0,0


In [ ]:
y_test

430     23082.95533
224     19515.54160
1103    11363.28320
1100    16776.30405
762     19040.87600
           ...     
1021    35595.58980
583     17626.23951
1206    36910.60803
618     34439.85590
876     26140.36030
Name: charges, Length: 268, dtype: float64

In [ ]:
y_train

335     13822.80300
662      5148.55260
3       21984.47061
530     48675.51770
712      8310.83915
           ...     
610      8547.69130
1106     8988.15875
1060     1981.58190
733      9447.25035
703      5385.33790
Name: charges, Length: 1070, dtype: float64

## Создаем и обучаем модель линейной регресии

In [ ]:
# Создайте и обучите модель
Lr=LinearRegression()

In [ ]:
Lr.fit(X_train,y_train)

LinearRegression()

In [ ]:
Lr.coef_

array([ 2.59224092e+02, -1.60871687e+01,  3.48480506e+02,  3.87358243e+02,
        2.38547486e+04,  5.49610035e+02,  3.61500330e+02, -2.57618008e+02,
       -6.53492357e+02])

## Получаем предсказание и оцениваем качество

In [ ]:
# Получите предсказание
Lr.predict(X_test)

array([ 2.84731288e+03,  3.01119448e+04,  1.43913987e+04,  2.74446353e+04,
        2.86276739e+04,  1.18181927e+04,  6.79154949e+03,  7.59449269e+03,
        5.50111388e+03,  1.08482257e+04,  2.78587897e+04,  6.41848933e+03,
        4.20489794e+03,  3.75571293e+02,  1.14412377e+04,  1.90497827e+03,
        2.81920741e+03,  9.20565499e+03,  1.70570226e+04,  1.06208037e+04,
        3.50513666e+04,  2.82203315e+03,  1.07953069e+04,  1.40448840e+04,
       -3.68179723e+01,  1.34516677e+04,  1.20897456e+04,  2.89361103e+04,
        5.06843731e+03,  2.27712011e+03,  3.83307032e+03,  2.73058471e+04,
        4.00687647e+03,  3.28296678e+04,  1.03772306e+04,  3.06411365e+03,
        3.68197821e+04,  1.92964925e+04,  1.09009842e+04,  7.90690129e+03,
        2.61047663e+04,  5.30985984e+03,  1.34549950e+04,  9.80194357e+03,
        1.29258555e+04,  1.26562377e+04,  1.05042908e+04,  1.08735044e+04,
        7.58973610e+03,  1.27275758e+04, -2.18176470e+03, -1.04701627e+01,
        3.09053721e+04,  

In [ ]:
## Проверка "на глаз"
pd.DataFrame([y_test.values, Lr.predict(X_test)]).T

,0,1
0,23082.95533,2847.312878
1,19515.54160,30111.944809
2,11363.28320,14391.398708
3,16776.30405,27444.635276
4,19040.87600,28627.673924
...,...,...
263,35595.58980,28328.930504
264,17626.23951,3723.405046
265,36910.60803,14599.497058
266,34439.85590,27117.507758


In [ ]:
# Оцените качество, при помощи метода mean_squared_error для тестовой выборки
mean_squared_error(y_test, Lr.predict(X_test))

40015277.56092456

Ошибка ~6325 долларов

In [ ]:
mean_squared_error(y_train, Lr.predict(X_train))

35696881.97594973

Ошибка ~5975. Не равно предыдущей ошибке, машина обучилась закономерностям

## Делаем предсказание для одного человека

In [ ]:
# Заполняем данные по конкретному человеку
data = [{
    "age": 20,
    "sex": 1,
    "bmi": 30,
    "children": 2,
    "smoker": 1,
    "region_northeast": 0,
    "region_northwest": 0,
    "region_southeast": 1,
    "region_southwest": 0
}]

In [ ]:
df_person = pd.DataFrame(data)
df_person.head()

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,20,1,30,2,1,0,0,1,0


In [ ]:
Lr.predict(df_person)

array([27051.5867928])